In [1]:

import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
import powerwall
import voltwatt


### Test Profiles

In [2]:

pw = powerwall.Powerwall()    # create EUT 

vw1 = voltwatt.VoltWattTest(1, pw)  # create test profiles
vw2 = voltwatt.VoltWattTest(2, pw)  # enter test characteristic (1,2,3)
vw3 = voltwatt.VoltWattTest(3, pw)

### Test Profiles to CSV

In [3]:
# make dataframe to hold test profiles
df1 = pd.DataFrame.from_dict(vw1.__dict__, orient='index')
df2 = pd.DataFrame.from_dict(vw2.__dict__, orient='index')
df3 = pd.DataFrame.from_dict(vw3.__dict__, orient='index')

VW = pd.DataFrame()
VW['vw.1'], VW['vw.2'], VW['vw.3'] = df1[0], df2[0], df3[0]
#print(VW)
VW.to_csv('out.csv')
VW.to_html('out.html')


### Calculate Watts as Function of Volts

In [4]:
# run voltage vector through volt watt function
data_length = 100
voltage_range_min = pw.vl
voltage_range_max = pw.vh

v = np.linspace(voltage_range_min, voltage_range_max, data_length)
wat1 = np.zeros(data_length)

for count, voltage in enumerate(v):

    wat1[count] = vw1.watts_from_volts(v[count])



### Ideal Volt Watt Curve

In [5]:
volt_watt_data = {'volts': v.T , 'watts': wat1.T}
vwdf = pd.DataFrame.from_dict(volt_watt_data)

### Tolerance range for test variables

In [6]:
watt_tolerance = pw.prated * 0.05
vwdf['w-hi'] = vwdf['watts'] + watt_tolerance
vwdf['w-lo'] = vwdf['watts'] - watt_tolerance

### Generate Expected Power from Voltage Steps

In [7]:
P = np.zeros(len(vw1.steps))
for count, voltage in enumerate(vw1.steps):
    P[count] = vw1.watts_from_volts(vw1.steps[count])

In [8]:
import plotly.graph_objs as go

df = vwdf
xvals = vwdf['volts']
yvals = vwdf['watts']

fig = go.Figure([
    go.Scatter(
        name='Expected P',
        x=xvals,
        y=yvals,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=xvals,
        y=yvals+watt_tolerance,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=xvals,
        y=yvals-watt_tolerance,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),

])

# Add test steps to plot
fig.add_trace(
    go.Scatter(
        name='Test Steps',
        mode='markers',
        x=vw1.steps,
        y=P,
        marker=dict(
            symbol='square',
            color='LightSkyBlue',
            size=5,
            line=dict(
                color='MediumPurple',
                width=2
            )
        ),
        showlegend=True
    )
)


fig.update_layout(
    yaxis_title='Real Power (W)',
    xaxis_title='Grid Voltage (V)',
    title = 'Volt-Watt Curve (characteristic 1)',
    hovermode="x"
)

fig.show()
fig.write_html("plot.html")